In [1]:
"""
This notebook demonstrates how to replicate converting tensorflow
weights from tensorflow's vggish to torchvggish
""" 

# Download the audioset directory using subversion
# !apt-get -qq install subversion   # uncomment if on linux
!svn checkout https://github.com/tensorflow/models/trunk/research/audioset

# Download audioset requirements
!pip install numpy scipy
!pip install resampy tensorflow six soundfile

# grab the VGGish model checkpoints & PCA params
!curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
!curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz

/bin/bash: svn: command not found
/bin/bash: curl: command not found
/bin/bash: curl: command not found


In [2]:
# Test install
# !mv audioset/* .
from vggish_smoke_test import *

2022-05-22 06:44:13.054816: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-22 06:44:13.054852: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



Testing your install of VGGish

Log Mel Spectrogram example:  [[-4.47303259 -4.29463765 -4.14939193 ... -3.97474254 -3.94778045
  -3.78685566]
 [-4.48592983 -4.28831745 -4.13994942 ... -3.98374974 -3.94981089
  -3.79512755]
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]
 ...
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]]


2022-05-22 06:44:16.917742: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-22 06:44:16.918976: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-22 06:44:16.919061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-22 06:44:16.919107: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-22 06:44:16.919151: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

INFO:tensorflow:Restoring parameters from /opt/ml/models/research/audioset/vggish/vggish_model.ckpt


2022-05-22 06:44:17.287349: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


VGGish embedding:  [-0.43252084 -0.25330514 -0.03891924 -0.16376027 -0.3499182  -0.59936893
 -0.05658104  0.16280285 -0.7555176  -0.08260237 -0.03138635 -0.83147156
 -0.10581692 -0.01420227 -0.1107798  -0.06599119 -0.22666278  0.8060121
 -0.56459844 -0.07349294 -0.06056742 -0.11864138 -0.26290444 -0.4155161
 -0.02423218  0.36676204  0.03564948 -0.549977   -0.00279108 -0.28981644
 -0.57134503  0.3810783   0.1366871   0.9188573   0.80642533 -0.05767322
 -0.13229543 -0.05044432 -0.22702815  0.04124349  0.7088706  -0.72661525
  0.4956671   0.24034092  0.21580261  0.88385975  1.1954073   0.6688216
  0.20919633  0.01531461  0.17449082 -0.6544126  -0.15788004  0.25017852
 -0.26469558 -0.3989996   0.14588487 -0.18502603  0.39927036  0.3041697
  0.1294817  -0.11220933 -0.4023689  -0.5374395  -0.36152244 -0.21291585
  0.5371816  -0.30606014 -0.08813701  0.04871783  0.42514458  0.18669182
 -0.17835425 -0.0693139   0.1470107  -0.2758583  -0.25766498  0.69777316
  0.45499614  0.05118209 -0.05679127

In [3]:
import tensorflow as tf
import vggish_slim

vggish_dict = {}
# load the model and get info 
with tf.Graph().as_default(), tf.Session() as sess:
    vggish_slim.define_vggish_slim(training=True)
    vggish_slim.load_vggish_slim_checkpoint(sess,"/opt/ml/pd/torchvggish/docs/vggish_model.ckpt")
    
    tvars = tf.trainable_variables()
    tvars_vals = sess.run(tvars)

    for var, val in zip(tvars, tvars_vals):
        print("%s" % (var.name))
        print("\t" + str(var.shape))
        vggish_dict[var.name] = val
    print("values written to vggish_dict")

AttributeError: module 'tensorflow' has no attribute 'Session'

In [ ]:
# Define torch model for vggish

import torch
import torch.nn as nn
import numpy as np

# From vggish_slim:
# The VGG stack of alternating convolutions and max-pools.
#     net = slim.conv2d(net, 64, scope='conv1')
#     net = slim.max_pool2d(net, scope='pool1')
#     net = slim.conv2d(net, 128, scope='conv2')
#     net = slim.max_pool2d(net, scope='pool2')
#     net = slim.repeat(net, 2, slim.conv2d, 256, scope='conv3')
#     net = slim.max_pool2d(net, scope='pool3')
#     net = slim.repeat(net, 2, slim.conv2d, 512, scope='conv4')
#     net = slim.max_pool2d(net, scope='pool4')
#     # Flatten before entering fully-connected layers
#     net = slim.flatten(net)
#     net = slim.repeat(net, 2, slim.fully_connected, 4096, scope='fc1')
#     # The embedding layer.
#     net = slim.fully_connected(net, params.EMBEDDING_SIZE, scope='fc2')

vggish_list = list(vggish_dict.values())
def param_generator():
    param = vggish_list.pop(0)
    transposed = np.transpose(param)
    to_torch = torch.from_numpy(transposed)
    result = torch.nn.Parameter(to_torch)
    yield result

class VGGish(nn.Module):
    def __init__(self):
        super(VGGish, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2))
        self.embeddings = nn.Sequential(
            nn.Linear(512*24, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 128),
            nn.ReLU(inplace=True))
        
        # extract weights from `vggish_list`
        for seq in (self.features, self.embeddings):
            for layer in seq:
                if type(layer).__name__ != "MaxPool2d" and type(layer).__name__ != "ReLU":
                    layer.weight = next(param_generator())
                    layer.bias = next(param_generator())
            
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0),-1)
        x = self.embeddings(x)
        return x

net = VGGish()
net.eval()

# Save weights to disk
torch.save(net.state_dict(), "./vggish.pth")